# Agent: Control

이 튜토리얼을 통해 다음을 배울 수 있다:

- AI 에이전트에서 제어 로직의 필요성을 이해한다.
- 의도 분류를 통한 라우팅을 구현한다.
- 조건부 흐름 제어로 예측 가능한 동작을 만든다.
- 다양한 제어 패턴을 실전에 적용한다.

## 1. 환경 설정

### 1.1 필요한 라이브러리 설치

In [ ]:
%pip install openai pydantic instructor python-dotenv

### 1.2 환경 설정

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()

MODEL="gpt-4o-mini"

### 1.3 필요한 라이브러리 가져오기

In [2]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Literal
import instructor

## 2. 기본 제어 구현

### 2.1 의도 분류 모델 정의
먼저 사용자 입력의 의도를 분류하는 모델을 정의한다.

In [6]:
client = instructor.patch(OpenAI())

class IntentClassification(BaseModel):
    """사용자 입력의 의도를 분류하기 위한 데이터 모델이다."""
    intent: Literal["question", "request", "complaint"] = Field(
        ..., description="사용자 입력의 의도. 'question', 'request', 'complaint' 중 하나다."
    )
    confidence: float = Field(..., description="분류의 신뢰도 점수 (0.0에서 1.0 사이).")
    reasoning: str = Field(..., description="이러한 의도로 분류한 이유에 대한 간략한 설명.")

### 2.2 핸들러 함수 정의
각 의도별로 처리할 핸들러 함수를 작성한다.

In [7]:
def answer_question(question: str) -> str:
    """질문에 답변하는 함수다."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "질문에 정확하고 간결하게 답변한다."},
            {"role": "user", "content": question}
        ]
    )
    return response.choices[0].message.content

def process_request(request: str) -> str:
    """요청을 처리하는 함수다."""
    return f"요청을 접수했습니다: {request}\n담당자가 곧 처리하겠습니다."

def handle_complaint(complaint: str) -> str:
    """불만 사항을 처리하는 함수다."""
    return f"불편을 드려 죄송합니다: {complaint}\n상급 부서로 즉시 전달하겠습니다."

### 2.3 라우팅 로직 구현
의도에 따라 적절한 핸들러로 요청을 전달한다.

In [8]:
def route_based_on_intent(user_input: str) -> tuple[str, IntentClassification]:
    """사용자 입력의 의도를 파악하여 적절한 핸들러로 라우팅하는 함수다."""
    
    classification = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "사용자 입력을 질문(question), 요청(request), 불만(complaint) 세 가지 범주 중 하나로 분류한다."
            },
            {"role": "user", "content": user_input}
        ],
        response_model=IntentClassification,
    )
    
    intent = classification.intent
    
    if intent == "question":
        result = answer_question(user_input)
    elif intent == "request":
        result = process_request(user_input)
    elif intent == "complaint":
        result = handle_complaint(user_input)
    else:
        result = "어떻게 도와드려야 할지 잘 모르겠습니다."
    
    return result, classification

# 사용 예시
test_inputs = [
    "머신러닝이란 무엇인가?",
    "내일 회의를 예약해줘.",
    "서비스 품질에 불만이 있다.",
]

for user_input in test_inputs:
    print(f"\n=== 입력: {user_input} ===")
    result, classification = route_based_on_intent(user_input)
    print(f"의도: {classification.intent} (신뢰도: {classification.confidence:.2f})")
    print(f"추론: {classification.reasoning}")
    print(f"응답: {result}")


=== 입력: 머신러닝이란 무엇인가? ===
의도: question (신뢰도: 0.95)
추론: 사용자가 '머신러닝'에 대한 정의를 요청하고 있어, 이는 정보 요청에 해당한다.
응답: 머신러닝은 컴퓨터가 데이터로부터 학습하고 경험을 바탕으로 예측이나 결정을 내릴 수 있게 하는 인공지능의 한 분야입니다. 이는 알고리즘과 통계 모델을 사용하여 패턴을 인식하고, 새로운 데이터에 대한 인사이트를 도출하는 과정입니다. 머신러닝은 일반적으로 지도학습, 비지도학습, 강화학습으로 나뉘며, 다양한 분야에서 활용됩니다.

=== 입력: 내일 회의를 예약해줘. ===
의도: request (신뢰도: 0.90)
추론: 사용자가 회의 예약을 요청하고 있기 때문에 요청으로 분류했습니다.
응답: 요청을 접수했습니다: 내일 회의를 예약해줘.
담당자가 곧 처리하겠습니다.

=== 입력: 서비스 품질에 불만이 있다. ===
의도: complaint (신뢰도: 0.95)
추론: 사용자가 서비스 품질에 대한 불만을 표현하고 있습니다.
응답: 불편을 드려 죄송합니다: 서비스 품질에 불만이 있다.
상급 부서로 즉시 전달하겠습니다.


## 3. 다양한 제어 패턴

### 3.1 딕셔너리 기반 라우팅
if/elif 대신 딕셔너리를 사용하여 더 확장 가능한 라우팅을 구현한다.

In [9]:
class ActionType(BaseModel):
    """작업 유형 분류 모델이다."""
    action: Literal["search", "create", "update", "delete", "read"]
    entity: str
    confidence: float

def search_handler(entity: str) -> str:
    """검색 작업을 처리한다."""
    return f"{entity}를 검색합니다."

def create_handler(entity: str) -> str:
    """생성 작업을 처리한다."""
    return f"새로운 {entity}를 생성합니다."

def update_handler(entity: str) -> str:
    """수정 작업을 처리한다."""
    return f"{entity}를 수정합니다."

def delete_handler(entity: str) -> str:
    """삭제 작업을 처리한다."""
    return f"{entity}를 삭제합니다."

def read_handler(entity: str) -> str:
    """조회 작업을 처리한다."""
    return f"{entity}를 조회합니다."

HANDLERS = {
    "search": search_handler,
    "create": create_handler,
    "update": update_handler,
    "delete": delete_handler,
    "read": read_handler,
}

def route_with_dict(user_input: str) -> str:
    """딕셔너리 기반 라우팅을 수행한다."""
    classification = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "사용자 요청에서 CRUD(search, create, update, delete, read) 작업 유형과 대상 엔티티를 파악한다."},
            {"role": "user", "content": user_input}
        ],
        response_model=ActionType,
    )
    
    handler = HANDLERS.get(classification.action)
    if handler:
        return handler(classification.entity)
    return "알 수 없는 작업입니다."

# 사용 예시
print(route_with_dict("고객 정보를 찾아줘"))
print(route_with_dict("새 제품을 등록해줘"))
print(route_with_dict("주문을 취소해줘"))

고객 정보를 검색합니다.
새로운 제품를 생성합니다.
주문를 삭제합니다.


### 3.2 우선순위 기반 라우팅
여러 조건을 우선순위에 따라 평가한다.

In [10]:
class Priority(BaseModel):
    """요청의 우선순위 모델이다."""
    level: Literal["critical", "high", "medium", "low"]
    is_urgent: bool
    requires_human: bool

def route_by_priority(user_input: str) -> str:
    """우선순위에 따라 라우팅한다."""
    priority = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "요청의 우선순위를 평가한다. 시스템 장애와 관련된 내용은 'critical'로 분류한다."},
            {"role": "user", "content": user_input}
        ],
        response_model=Priority,
    )
    
    if priority.level == "critical":
        return "긴급 대응팀에 즉시 연결합니다."
    elif priority.requires_human:
        return "상담원 연결을 준비합니다."
    elif priority.is_urgent:
        return "우선 처리 대기열에 추가합니다."
    else:
        return "일반 처리 절차를 진행합니다."

# 사용 예시
print(f'입력: 시스템이 다운됐어요! -> 처리: {route_by_priority("시스템이 다운됐어요!")}')
print(f'입력: 계정 삭제를 원합니다 -> 처리: {route_by_priority("계정 삭제를 원합니다")}')
print(f'입력: 제품 정보가 궁금해요 -> 처리: {route_by_priority("제품 정보가 궁금해요")}')

입력: 시스템이 다운됐어요! -> 처리: 긴급 대응팀에 즉시 연결합니다.
입력: 계정 삭제를 원합니다 -> 처리: 상담원 연결을 준비합니다.
입력: 제품 정보가 궁금해요 -> 처리: 일반 처리 절차를 진행합니다.


### 3.3 상태 기반 제어
현재 상태에 따라 다른 동작을 수행한다. 이는 유한 상태 머신(Finite State Machine)의 간단한 형태다.

In [11]:
class ConversationState:
    """대화 상태를 관리하는 클래스다."""
    
    def __init__(self):
        self.state = "initial"
        self.collected_info = {}
    
    def process(self, user_input: str) -> str:
        """상태에 따라 다르게 처리한다."""
        
        if self.state == "initial":
            self.state = "collecting_name"
            return "안녕하세요! 먼저 성함을 알려주시겠어요?"
        
        elif self.state == "collecting_name":
            self.collected_info["name"] = user_input
            self.state = "collecting_email"
            return f"{user_input}님, 반갑습니다! 이메일 주소를 알려주세요."
        
        elif self.state == "collecting_email":
            self.collected_info["email"] = user_input
            self.state = "collecting_issue"
            return "문의 사항을 말씀해 주세요."
        
        elif self.state == "collecting_issue":
            self.collected_info["issue"] = user_input
            self.state = "completed"
            return self._generate_summary()
        
        else:
            return "문의가 접수되었습니다. 감사합니다!"
    
    def _generate_summary(self) -> str:
        """수집된 정보를 요약한다."""
        return f"""
문의 접수 완료:
- 이름: {self.collected_info['name']}
- 이메일: {self.collected_info['email']}
- 문의: {self.collected_info['issue']}
담당자가 곧 연락드리겠습니다.
"""

# 사용 예시
conversation = ConversationState()
print(f"AI: {conversation.process('안녕하세요')}")
print(f"AI: {conversation.process('김철수')}")
print(f"AI: {conversation.process('chulsoo@example.com')}")
print(f"AI: {conversation.process('제품 배송이 지연되고 있어요')}")

AI: 안녕하세요! 먼저 성함을 알려주시겠어요?
AI: 김철수님, 반갑습니다! 이메일 주소를 알려주세요.
AI: 문의 사항을 말씀해 주세요.
AI: 
문의 접수 완료:
- 이름: 김철수
- 이메일: chulsoo@example.com
- 문의: 제품 배송이 지연되고 있어요
담당자가 곧 연락드리겠습니다.



## 4. 실전 예제

### 4.1 고객 지원 라우팅 시스템
복잡한 고객 지원 시나리오를 처리한다.

In [12]:
from typing import Optional

class CustomerIntent(BaseModel):
    """고객 의도 분류 모델이다."""
    category: Literal["technical", "billing", "general", "complaint"]
    subcategory: Optional[str]
    urgency: Literal["low", "medium", "high"]
    sentiment: Literal["positive", "neutral", "negative"]

def route_customer_support(user_input: str) -> str:
    """고객 지원 요청을 라우팅한다."""
    intent = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "고객 문의를 분석하여 기술, 결제, 일반, 불만 카테고리로 분류하고 긴급도와 감정을 평가한다."},
            {"role": "user", "content": user_input}
        ],
        response_model=CustomerIntent,
    )
    
    if intent.urgency == "high":
        return f"[긴급] {intent.category} 팀에 즉시 전달합니다."
    
    if intent.category == "technical":
        return "기술 지원팀으로 연결합니다. 잠시만 기다려주세요."
    
    elif intent.category == "billing":
        return "결제 관련 문의는 재무팀에서 처리합니다."
    
    elif intent.category == "complaint":
        return "고객만족팀에서 신속히 처리하겠습니다."
    
    else:
        return "일반 상담팀에서 도와드리겠습니다."

# 사용 예시
queries = [
    "로그인이 안 돼요! 빨리 해결해주세요!",
    "청구서에 오류가 있는 것 같아요",
    "제품 사용법을 알고 싶어요",
]

for query in queries:
    print(f"\n문의: {query}")
    print(f"처리: {route_customer_support(query)}")


문의: 로그인이 안 돼요! 빨리 해결해주세요!
처리: [긴급] technical 팀에 즉시 전달합니다.

문의: 청구서에 오류가 있는 것 같아요
처리: 결제 관련 문의는 재무팀에서 처리합니다.

문의: 제품 사용법을 알고 싶어요
처리: 일반 상담팀에서 도와드리겠습니다.


### 4.2 작업 흐름 오케스트레이션
여러 단계의 작업을 순차적으로 관리한다.

In [13]:
class WorkflowOrchestrator:
    """작업 흐름을 관리하는 오케스트레이터다."""
    
    def __init__(self):
        self.steps = ["validate", "process", "approve", "complete"]
        self.current_step_index = 0
        self.data = {}
    
    def execute_next_step(self, input_data: str) -> str:
        """다음 단계를 실행한다."""
        if self.current_step_index >= len(self.steps):
            return self._complete()
            
        step_name = self.steps[self.current_step_index]
        
        if step_name == "validate":
            result = self._validate(input_data)
        elif step_name == "process":
            result = self._process(input_data)
        elif step_name == "approve":
            result = self._approve(input_data)
        else:
            return "알 수 없는 단계입니다."
        
        self.current_step_index += 1
        return result
    
    def _validate(self, data: str) -> str:
        """데이터를 검증한다."""
        self.data["validated_data"] = data
        return f"[1/4] 검증 완료: {data}"
    
    def _process(self, data: str) -> str:
        """데이터를 처리한다."""
        self.data["processed_data"] = data
        return f"[2/4] 처리 완료: {data}"
    
    def _approve(self, data: str) -> str:
        """승인을 진행한다."""
        self.data["approval_info"] = data
        return f"[3/4] 승인 완료: {data}"
    
    def _complete(self) -> str:
        """작업을 완료한다."""
        return "[4/4] 모든 단계가 완료되었습니다."

# 사용 예시
workflow = WorkflowOrchestrator()
print(workflow.execute_next_step("주문 데이터"))
print(workflow.execute_next_step("결제 정보"))
print(workflow.execute_next_step("관리자 확인"))
print(workflow.execute_next_step("최종 실행"))

[1/4] 검증 완료: 주문 데이터
[2/4] 처리 완료: 결제 정보
[3/4] 승인 완료: 관리자 확인
알 수 없는 단계입니다.


### 4.3 다중 에이전트 라우팅
전문화된 에이전트들 간의 라우팅을 구현한다.

In [14]:
class AgentType(BaseModel):
    """에이전트 유형 선택 모델이다."""
    agent: Literal["researcher", "writer", "coder", "analyst"]
    task_description: str

class MultiAgentRouter:
    """여러 전문 에이전트로 라우팅하는 시스템이다."""
    
    def __init__(self):
        self.agents = {
            "researcher": self._research_agent,
            "writer": self._writer_agent,
            "coder": self._coder_agent,
            "analyst": self._analyst_agent,
        }
    
    def route(self, task: str) -> str:
        """작업을 적절한 에이전트로 라우팅한다."""
        selection = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": "주어진 작업에 가장 적합한 전문 에이전트(researcher, writer, coder, analyst)를 선택한다."},
                {"role": "user", "content": task}
            ],
            response_model=AgentType,
        )
        
        agent_func = self.agents.get(selection.agent)
        if agent_func:
            return agent_func(selection.task_description)
        return "적절한 에이전트를 찾을 수 없습니다."
    
    def _research_agent(self, task: str) -> str:
        """리서치 에이전트가 작업을 수행한다."""
        return f"[리서치 에이전트] '{task}' 관련 자료를 조사합니다."
    
    def _writer_agent(self, task: str) -> str:
        """작가 에이전트가 작업을 수행한다."""
        return f"[작가 에이전트] '{task}' 관련 글을 작성합니다."
    
    def _coder_agent(self, task: str) -> str:
        """코더 에이전트가 작업을 수행한다."""
        return f"[코더 에이전트] '{task}' 관련 코드를 작성합니다."
    
    def _analyst_agent(self, task: str) -> str:
        """분석가 에이전트가 작업을 수행한다."""
        return f"[분석가 에이전트] '{task}' 관련 데이터를 분석합니다."

# 사용 예시
router = MultiAgentRouter()
tasks = [
    "AI 트렌드에 대한 최신 정보를 찾아줘",
    "마케팅 블로그 포스트를 작성해줘",
    "웹 스크래퍼를 만들어줘",
    "매출 데이터를 분석해줘",
]

for task in tasks:
    print(f"\n작업: {task}")
    print(router.route(task))


작업: AI 트렌드에 대한 최신 정보를 찾아줘
[리서치 에이전트] 'Find the latest information and trends related to artificial intelligence (AI) for 2023.' 관련 자료를 조사합니다.

작업: 마케팅 블로그 포스트를 작성해줘
[작가 에이전트] 'Write a marketing blog post.' 관련 글을 작성합니다.

작업: 웹 스크래퍼를 만들어줘
[코더 에이전트] 'Create a web scraper that can extract data from a website.' 관련 코드를 작성합니다.

작업: 매출 데이터를 분석해줘
[분석가 에이전트] '매출 데이터를 분석해줘' 관련 데이터를 분석합니다.


## 5. 제어 패턴 모범 사례

### 5.1 명확한 분류 기준
의도나 카테고리를 명확하게 정의하여 LLM의 모호한 판단을 줄인다.

In [15]:
class WellDefinedIntent(BaseModel):
    """명확하게 정의된 의도 모델이다."""
    intent: Literal[
        "product_inquiry",      # 제품 문의
        "order_status",         # 주문 상태 확인
        "return_request",       # 반품 요청
        "technical_support",    # 기술 지원
        "account_management"    # 계정 관리
    ]
    description: str = Field("각 의도의 명확한 정의를 제공한다")

### 5.2 폴백(Fallback) 처리
예상치 못한 경우나 분류에 실패한 경우를 처리하는 기본 동작(`get` 메서드의 기본값 등)을 정의한다.

### 5.3 로깅 및 모니터링
라우팅 결정을 기록하여 시스템의 동작을 추적하고 디버깅할 수 있게 한다.

In [16]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def route_with_logging(user_input: str) -> str:
    """로깅이 포함된 라우팅 함수다."""
    logger.info(f"입력 수신: {user_input}")
    classification = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "사용자 입력을 질문, 요청, 불만으로 분류한다."},
            {"role": "user", "content": user_input}
        ],
        response_model=IntentClassification,
    )
    
    logger.info(f"의도 분류: {classification.intent}, 신뢰도: {classification.confidence:.2f}")
    
    handlers = {
        "question": answer_question,
        "request": process_request,
        "complaint": handle_complaint,
    }
    
    handler = handlers.get(classification.intent, lambda x: "알 수 없는 의도입니다.")
    result = handler(user_input)
    
    logger.info(f"응답 생성: {result[:50]}...")
    return result

# 사용 예시
route_with_logging("환불 정책에 대해 알려주세요.")

INFO:__main__:입력 수신: 환불 정책에 대해 알려주세요.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:의도 분류: question, 신뢰도: 0.95
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:응답 생성: 환불 정책은 판매자나 서비스 제공자가 고객의 구매 후, 특정 조건을 충족할 경우 금액을 돌...


'환불 정책은 판매자나 서비스 제공자가 고객의 구매 후, 특정 조건을 충족할 경우 금액을 돌려주는 규정을 말합니다. 일반적으로 환불 정책의 주요 요소는 다음과 같습니다:\n\n1. **환불 기간**: 구매 후 언제까지 환불 요청이 가능한지 명시.\n2. **상태 조건**: 환불이 가능한 제품의 상태(예: 사용되지 않은 제품, 원래 포장 유지 등).\n3. **환불 방법**: 환불이 어떻게 이루어지는지(예: 원래 결제 방법으로, 상품 교환 등).\n4. **면책 사항**: 환불이 불가능한 경우나 조건.\n\n각 기업마다 환불 정책은 상이하므로, 구매 전 반드시 확인하는 것이 중요합니다.'